In [0]:
# Cell 1: Read Silver Claims and start Gold aggregations
from pyspark.sql.functions import col, count, sum, avg, current_timestamp, year, month, quarter

print("📊 GOLD LAYER: CLAIMS AGGREGATIONS")
print("=" * 80)

# Read cleaned Silver claims
silver_claims = spark.table("smart_claims_dev.silver.claims_clean")

print(f"📈 Silver Claims Input: {silver_claims.count():,} rows")
print("\n🔍 Schema:")
silver_claims.printSchema()

In [0]:
# Cell 2: Claims aggregation by date and severity
from pyspark.sql.functions import col, count, sum, avg, year, month, date_format, current_timestamp

print("📊 GOLD: CLAIMS BY DATE & SEVERITY")
print("=" * 80)

# Read Silver claims
silver_claims = spark.table("smart_claims_dev.silver.claims_clean")

# Aggregate claims by date and severity
claims_by_date_severity = silver_claims.groupBy(
    date_format(col("claim_date"), "yyyy-MM").alias("claim_month"),
    col("severity")
).agg(
    count("claim_no").alias("claim_count"),
    avg(col("total")).alias("avg_payout"),
    sum(col("total")).alias("total_payout"),
    avg(col("age")).alias("avg_customer_age"),
    sum(col("injury")).alias("total_injury_claims"),
    sum(col("property")).alias("total_property_claims"),
    sum(col("vehicle")).alias("total_vehicle_claims")
).withColumn("processed_at", current_timestamp())

print(f"✅ Created: claims_by_date_severity")
print(f"   Rows: {claims_by_date_severity.count():,}")

# Show sample
print("\n🔍 SAMPLE DATA:")
claims_by_date_severity.show(10, truncate=False)

# Write to Gold layer
claims_by_date_severity.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("smart_claims_dev.gold.claims_by_date_severity")

print("\n✅ Written to: smart_claims_dev.gold.claims_by_date_severity")


In [0]:
# Cell 3: Claims aggregation by collision type
from pyspark.sql.functions import col, count, sum, avg, current_timestamp

print("📊 GOLD: CLAIMS BY COLLISION TYPE")
print("=" * 80)

# Read Silver claims
silver_claims = spark.table("smart_claims_dev.silver.claims_clean")

# Aggregate by collision type
claims_by_collision = silver_claims.groupBy(
    col("collision_type")
).agg(
    count("claim_no").alias("claim_count"),
    avg(col("total")).alias("avg_payout"),
    sum(col("total")).alias("total_payout"),
    avg(col("number_of_vehicles_involved")).alias("avg_vehicles"),
    sum(col("number_of_witnesses")).alias("total_witnesses"),
    sum(col("suspicious_activity").cast("int")).alias("suspicious_claims")
).withColumn("processed_at", current_timestamp())

print(f"✅ Created: claims_by_collision")
print(f"   Rows: {claims_by_collision.count():,}")

# Show sample
print("\n🔍 SAMPLE DATA:")
claims_by_collision.show(10, truncate=False)

# Write to Gold layer
claims_by_collision.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("smart_claims_dev.gold.claims_by_collision_type")

print("\n✅ Written to: smart_claims_dev.gold.claims_by_collision_type")

In [0]:
# Cell 4: Customer behavior metrics
from pyspark.sql.functions import col, count, sum, avg, current_timestamp

print("📊 GOLD: CUSTOMER CLAIMS METRICS")
print("=" * 80)

# Read Silver claims
silver_claims = spark.table("smart_claims_dev.silver.claims_clean")

# Aggregate by customer
customer_metrics = silver_claims.groupBy(
    col("policy_no").alias("customer_id")
).agg(
    count("claim_no").alias("total_claims"),
    sum(col("total")).alias("total_payout"),
    avg(col("total")).alias("avg_claim_amount"),
    sum(col("injury")).alias("injury_claims"),
    sum(col("property")).alias("property_claims"),
    sum(col("vehicle")).alias("vehicle_claims"),
    sum(col("suspicious_activity").cast("int")).alias("suspicious_claims")
).withColumn("processed_at", current_timestamp())

count_before = customer_metrics.count()
print(f"✅ Aggregated: {count_before:,} customers")

# WRITE
customer_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("smart_claims_dev.gold.customer_metrics")

# VERIFY
count_after = spark.table("smart_claims_dev.gold.customer_metrics").count()
print(f"✅ Written and verified: {count_after:,} rows in smart_claims_dev.gold.customer_metrics")
print("=" * 80)

In [0]:
# Cell 5: Telematics driver behavior scoring
from pyspark.sql.functions import col, count, sum, avg, max, current_timestamp, when

print("📊 GOLD: TELEMATICS DRIVER SCORING")
print("=" * 80)

# Read Silver telematics
silver_telematics = spark.table("smart_claims_dev.silver.telematics_clean")

# Classify risky driving events
telematics_scored = silver_telematics.withColumn(
    "high_speed_event",
    when(col("speed_mph") > 80, 1).otherwise(0)
).withColumn(
    "harsh_acceleration_event",
    when(col("acceleration_mps2") > 5, 1).otherwise(0)
).withColumn(
    "harsh_braking_event",
    when(col("acceleration_mps2") < -5, 1).otherwise(0)
)

# Aggregate by vehicle
driver_risk_scores = telematics_scored.groupBy(
    col("vehicle_id").alias("vehicle_id")
).agg(
    count("timestamp").alias("total_events"),
    sum(col("high_speed_event")).alias("high_speed_count"),
    sum(col("harsh_acceleration_event")).alias("harsh_accel_count"),
    sum(col("harsh_braking_event")).alias("harsh_brake_count"),
    avg(col("speed_mph")).alias("avg_speed"),
    max(col("speed_mph")).alias("max_speed"),
    avg(col("acceleration_mps2")).alias("avg_acceleration")
).withColumn(
    "risk_score",
    (col("high_speed_count") * 0.3 + 
     col("harsh_accel_count") * 0.2 + 
     col("harsh_brake_count") * 0.2)
).withColumn("processed_at", current_timestamp())

print(f"✅ Created: driver_risk_scores")
print(f"   Rows: {driver_risk_scores.count():,}")

# Show sample
print("\n🔍 SAMPLE DATA (Top 10 by risk score):")
driver_risk_scores.orderBy(col("risk_score").desc()).show(10, truncate=False)

# Write to Gold layer
driver_risk_scores.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("smart_claims_dev.gold.driver_risk_scores")

print("\n✅ Written to: smart_claims_dev.gold.driver_risk_scores")

In [0]:
# Cell 6A: Inspect Silver Policies
from pyspark.sql.functions import col

print("🔍 DEBUGGING: Silver Policies Data")
print("=" * 80)

silver_policies = spark.table("smart_claims_dev.silver.policies_clean")

# Check schema
print("📊 Schema:")
silver_policies.printSchema()

# Show raw values of problematic columns
print("\n📋 Sample SUM_INSURED values (raw):")
silver_policies.select("POLICY_NO", "SUM_INSURED").show(10, truncate=False)

print("\n📋 Sample PREMIUM values (raw):")
silver_policies.select("POLICY_NO", "PREMIUM").show(10, truncate=False)

# Check data types
print("\n📊 Column types:")
print(f"SUM_INSURED type: {silver_policies.schema['SUM_INSURED'].dataType}")
print(f"PREMIUM type: {silver_policies.schema['PREMIUM'].dataType}")

In [0]:
# Cell 6: ML feature engineering - NO JOINS, just claims data
from pyspark.sql.functions import col, when, current_timestamp

print("📊 GOLD: ML FEATURE ENGINEERING")
print("=" * 80)

# Read ONLY Silver claims
silver_claims = spark.table("smart_claims_dev.silver.claims_clean")

# Feature engineering - NO JOINS
ml_features_engineered = silver_claims.select(
    col("claim_no").alias("claim_id"),
    col("policy_no"),
    col("age"),
    col("total").alias("claim_amount"),
    col("months_as_customer"),
    col("number_of_vehicles_involved"),
    col("number_of_witnesses"),
    col("suspicious_activity").cast("int").alias("suspicious_flag"),
    
    # Fraud indicators
    when(col("suspicious_activity") == True, 1).otherwise(0).alias("fraud_indicator"),
    when(col("number_of_witnesses") == 0, 1).otherwise(0).alias("no_witnesses_flag"),
    when(col("months_as_customer") < 6, 1).otherwise(0).alias("new_customer_flag"),
    
    # Claim severity features
    when(col("severity") == "Total Loss", 1).otherwise(0).alias("total_loss_flag"),
    when(col("severity") == "Major", 1).otherwise(0).alias("major_damage_flag"),
    when(col("number_of_vehicles_involved") > 2, 1).otherwise(0).alias("multi_vehicle_flag")
).withColumn("processed_at", current_timestamp())

print(f"✅ Created: ml_features_engineered")
print(f"   Rows: {ml_features_engineered.count():,}")

# Show sample
print("\n🔍 SAMPLE ML FEATURES (First 5 rows):")
ml_features_engineered.show(5, truncate=False)

# Write to Gold layer
ml_features_engineered.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("smart_claims_dev.gold.ml_features")

print("\n✅ Written to: smart_claims_dev.gold.ml_features")

# Show data types
print("\n📊 ML FEATURES SCHEMA:")
ml_features_engineered.printSchema()

In [0]:
# Cell 7: Gold Layer Summary - All tables complete
from pyspark.sql.functions import col, count
import time

print("🏆 GOLD LAYER COMPLETE")
print("=" * 80)

time.sleep(1)  # delay for table availability

# Read all Gold tables
gold_claims_date_severity = spark.table("smart_claims_dev.gold.claims_by_date_severity")
gold_claims_collision = spark.table("smart_claims_dev.gold.claims_by_collision_type")
gold_customer_metrics = spark.table("smart_claims_dev.gold.customer_metrics")
gold_driver_scores = spark.table("smart_claims_dev.gold.driver_risk_scores")
gold_ml_features = spark.table("smart_claims_dev.gold.ml_features")

print("\n📊 GOLD LAYER TABLE SUMMARY:")
print("=" * 80)
print(f"✅ claims_by_date_severity:    {gold_claims_date_severity.count():>10,} rows")
print(f"✅ claims_by_collision_type:   {gold_claims_collision.count():>10,} rows")
print(f"✅ customer_metrics:           {gold_customer_metrics.count():>10,} rows")
print(f"✅ driver_risk_scores:         {gold_driver_scores.count():>10,} rows")
print(f"✅ ml_features:                {gold_ml_features.count():>10,} rows")
print("=" * 80)

print("\n📈 MEDALLION ARCHITECTURE COMPLETE:")
print("=" * 80)
print("Bronze Layer:  37,289 raw records")
print("Silver Layer:  31,504 validated records (84.5% quality)")
print("Gold Layer:    5 analytics & ML-ready tables")
print("=" * 80)

print("\n🎯 PROJECT STATUS: PRODUCTION-READY END-TO-END PIPELINE ✅")
print("=" * 80)